Ucitavamo dataset

In [66]:
import numpy as np 
ulaz= np.loadtxt('TestDataset.txt')

In [68]:
import torch
import numpy as np 
from torch.utils.data import TensorDataset



ulaz= np.loadtxt('TrainDataset.txt')
podaci=ulaz[:,1:]
obelezja=ulaz[:,:1]
obelezja=torch.Tensor(obelezja)
obelezja=obelezja.flatten()

ulaz= np.loadtxt('TestDataset.txt')
tpodaci=ulaz[:,1:]
tmp = np.concatenate((podaci,tpodaci))
b=np.max(tmp,axis=0)
podaci=podaci/b
tpodaci=tpodaci/b
tobelezja=ulaz[:,:1]
tpodaci=torch.Tensor(tpodaci)
tobelezja=torch.Tensor(tobelezja)
tobelezja=tobelezja.flatten()


podaci=torch.Tensor(podaci)

test_set=TensorDataset(tpodaci,tobelezja)
dataset=TensorDataset(podaci,obelezja)
train_set,val_set = torch.utils.data.random_split(dataset,[0.9,0.1])

In [69]:
print(len(train_set))
print(len(val_set))
print(len(test_set))
print(len(b))

112500
12500
24998
1564


In [70]:
print(test_set[24100])

(tensor([0.5020, 0.4962, 0.4383,  ..., 0.0000, 0.0536, 0.5730]), tensor(48.))


In [71]:
if torch.cuda.is_available():
    device="cuda"
else:
    device="cpu"
    print("NE RADI CUDA!")

NE RADI CUDA!


Inicijalizacija mreze

In [72]:
import torch.nn as nn

class Mreza (nn.Module):
    def __init__ (self):
        super().__init__()
        self.act=nn.Sequential(
            nn.Linear(1564,1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(),
            nn.Linear(1000,300),
            nn.BatchNorm1d(300),
            nn.ReLU(),
            nn.Linear(300,120),
            nn.BatchNorm1d(120),
            nn.ReLU(),
            nn.Linear(120,50),
            nn.BatchNorm1d(50),
            nn.ReLU()
        )
        self.act=self.act.to(device)
    def forward (self, x):
        return self.act(x)
model = Mreza()
model = model.to(device)

Testiranje

In [73]:
def totensor(y):
    ret=np.zeros((len(y),50))
   # print(y[0])
   # print(y[0][0])
   # print(y[0][0].item())
    for i in range(0,len(y)):
        ret[i][round(y[i].item())]=1
    return torch.Tensor(ret).to(device)

def test(model,sample):
    #print(sample)
    sample=sample.to(device)
    rez=model(sample.reshape(1,1564))
    _,b=torch.max(rez,dim=1)
    return b


Inicijalizacija parametara

In [74]:
broj_epoha = 15
batch_velicina=500
loss_funk = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_velicina,shuffle=True,num_workers=0,drop_last=True,pin_memory=True)
val_loader = torch.utils.data.DataLoader(dataset=val_set,batch_size=batch_velicina,shuffle=False,num_workers=0,drop_last=False,pin_memory=True)


In [75]:
def get_accuracy(y_hat,y):
   # print(y)
    y_hat = y_hat.cpu().detach().numpy()
    y=y.cpu().numpy()
    y_hat=np.argmax(y_hat,axis=1)
   # print(y_hat,y)
    corrects=(y_hat==y).sum()
   # print(corrects)
    return corrects.item()

def validation(model):
    with torch.no_grad():
        model.eval()
        kol=0
        total_loss=0
        for batch_num, (x, y) in enumerate(val_loader):
                x = x.to(device)
                y = y.to(device)
                #print(y)
                y_hat = model(x)
                loss=loss_funk(y_hat,totensor(y))
                total_loss+=loss.detach().item()
                kol+=get_accuracy(y_hat=y_hat, y=y)/len(y)
                del y_hat
        kol=kol*100/batch_num
        total_loss/=batch_num
    return total_loss,kol

Treniranje

In [76]:
# MAKE SURE TO INITIALIZE PARAMETERS BEFORE STARTING TRAINING
best_model=model
best_loss=100000000

import copy
import time
for epoha in range(1,broj_epoha+1):
    train_running_loss =0.0
    train_acc=0.0
    model = model.train() 
    start=time.time()
    for batch_num, (x, y) in enumerate(loader):
            x = x.to(device)
            y = y.to(device)
           # print(y)
            #print(y)
            y_hat = model(x)
            loss = loss_funk(y_hat, totensor(y))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            train_running_loss += loss.detach().item()
            train_acc += get_accuracy(y_hat=y_hat, y=y)/len(y)
            del y_hat

            print(f'Epoch: {epoha}\t{100 * (batch_num + 1) / len(loader):.2f}% complete. {time.time() - start:.2f} seconds elapsed in epoch. Time until completion {(time.time()-start)/((batch_num + 1) / len(loader)) - time.time()+start:.2f} seconds.',
                end='\r')

    print(f'',end='\n')    
    epoch_loss = train_running_loss / batch_num
    epoch_acc = 100*train_acc / batch_num
    val_loss,val_acc = validation(model)
    if val_loss<best_loss:
          best_loss=val_loss
          best_model=copy.deepcopy(model)
    print('Epoch: %d | Train Loss: %.4f | Train Accuracy: %.2f | Validation Loss: %.4f | Validation Accuracy: %.2f' \
          %(epoha, epoch_loss, epoch_acc,val_loss,val_acc))

Epoch: 1	100.00% complete. 46.53 seconds elapsed in epoch. Time until completion 0.00 seconds.
Epoch: 1 | Train Loss: 3.4350 | Train Accuracy: 13.89 | Validation Loss: 3.3682 | Validation Accuracy: 18.18
Epoch: 2	100.00% complete. 43.89 seconds elapsed in epoch. Time until completion 0.00 seconds.
Epoch: 2 | Train Loss: 3.0520 | Train Accuracy: 21.69 | Validation Loss: 3.1845 | Validation Accuracy: 22.23
Epoch: 3	100.00% complete. 44.34 seconds elapsed in epoch. Time until completion 0.00 seconds.
Epoch: 3 | Train Loss: 2.8410 | Train Accuracy: 26.23 | Validation Loss: 3.0902 | Validation Accuracy: 23.76
Epoch: 4	100.00% complete. 45.41 seconds elapsed in epoch. Time until completion 0.00 seconds.
Epoch: 4 | Train Loss: 2.6640 | Train Accuracy: 30.26 | Validation Loss: 3.0588 | Validation Accuracy: 25.13
Epoch: 5	100.00% complete. 48.22 seconds elapsed in epoch. Time until completion 0.00 seconds.
Epoch: 5 | Train Loss: 2.4965 | Train Accuracy: 34.23 | Validation Loss: 2.9952 | Validat

In [ ]:
def testt(model):
        kol=0
        for i in range(0,len(val_set)):
            out=test(model,val_set[i][0].reshape(1,1564))
            ans=val_set[i][1].item()
            if round(ans)==out:
                kol+=1
        return kol
print(testt(model))
print(testt(model)/len(val_set)*100)

In [77]:
print(best_loss)
print(validation(best_model))

2.995218594868978
(2.995218594868978, 25.758333333333344)


In [ ]:
import torch
import numpy as np 
from torch.utils.data import TensorDataset

ulaz= np.loadtxt('TestDataset.txt')
podaci=ulaz[:,1:]
#print(podaci)
b=np.max(podaci,axis=0)
podaci=podaci/b
#print(podaci)
#print(np.max(podaci),np.min(podaci))
obelezja=ulaz[:,:1]
#print(obelezja)
podaci=torch.Tensor(podaci)
obelezja=torch.Tensor(obelezja)
obelezja=obelezja.flatten()

test_set=TensorDataset(podaci,obelezja)
print(len(test_set))

In [78]:
#ouf = open("Rezultati.txt")
with torch.no_grad():
    model.eval()
    kol=0
    for i in range(0,len(test_set)):
        if(test_set[i][1].cpu().item()==test(model,test_set[i][0]).cpu().item()):
            kol+=1
    print(kol)

5871


### OBRADA REZULTATA

In [83]:
gmatrix = []
for i in range(0,len(test_set)):
    a=test_set[i][0].to(device)
    guess=test(model,a)
    gmatrix.append(guess.item())
gmatrix=np.array(gmatrix)

In [84]:
labele=[]
for i in range(0,len(test_set)):
    labele.append(test_set[i][1].item())
labele=np.array(labele)
print(gmatrix,labele)

(tensor([0.2337, 0.2610, 0.2828,  ..., 0.0000, 0.0000, 0.1685]), tensor(0.))


[23 23 17 ... 43 49 43] [ 0.  0.  0. ... 49. 49. 49.]


In [87]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(labele, gmatrix)

for i in range(0, 50):
    p = np.sum(cf_matrix[i])
    cf_matrix[i] = cf_matrix[i] * 100 / p

plt.figure(figsize=(160, 90))
sns.heatmap(cf_matrix/100, annot=True, fmt='.2%', cmap='Blues')
plt.show()

In [88]:
torch.save(best_model,"BestModelFCC.pt")